## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham".

### Данные

In [ ]:
import pandas as pd
df = pd.read_csv('3_data (1).csv', encoding='latin-1')

In [ ]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df = df.drop_duplicates('text')

In [ ]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

<ipython-input-22-3d7c89935395>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].map({'ham': 0, 'spam': 1})


### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re

# stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))
snowball = SnowballStemmer(language="english")
def preprocess(text):
    no_stops = []
    text_finally =''
    text = re.sub(r'[^\w\s]', '', text.lower())
    text_split = text.split()
    for t in text_split:
      if t not in stopwords:
          no_stops.append(snowball.stem(t))
          text_finally +=snowball.stem(t)+' '
    return text_finally[:(len(text_finally)-1)]

In [ ]:
preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...")

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [ ]:
preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

'im gonna home soon dont want talk stuff anymor tonight k ive cri enough today'

Проверка, что функция работает верно

In [ ]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [ ]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [ ]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=90)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 90, C = 1.5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.998     0.988      1114
           1      0.987     0.855     0.916       179

    accuracy                          0.978      1293
   macro avg      0.982     0.926     0.952      1293
weighted avg      0.979     0.978     0.978      1293



Выполним предсказание для конкретного текста

In [ ]:
txt = "I think this book is a must read for anyone who wants an insight into the Middle East."
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [ ]:
model.predict(txt)

array([0])